In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir=(f"/content/drive/MyDrive/Major/flowers/flowers")

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow_hub as hub
from keras.layers import Flatten,Dense, Conv2D, AveragePooling2D, Dropout, MaxPool2D, Rescaling ,BatchNormalization, Activation
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
#import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)
train_datagen=ImageDataGenerator(rescale=1/255.,
                                 validation_split= 0.2
                                 )
# test_datagen =ImageDataGenerator(rescale=1/255.)

train_data= train_datagen.flow_from_directory(train_dir,target_size=(224,224), subset="training")
test_data= train_datagen.flow_from_directory(train_dir,target_size=(224,224), subset="validation", shuffle=False)

In [ ]:
x= train_data.next()
for i in range(18,22):
  image=x[0][i]
  plt.imshow(image)
  plt.show()

In [ ]:
resnet_url="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

In [ ]:
IMG_SHAPE=(224,224)
model_resnet=tf.keras.models.Sequential([
                         hub.KerasLayer(resnet_url,
                                        trainable=False,
                                        name="feature_extractor_layer",
                                        input_shape=IMG_SHAPE+(3,)),
                         Dense(5,activation="softmax",name="output_layer")
])

In [ ]:
model_resnet.summary()

In [ ]:
earlystopping=tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                               min_delta =1e-4,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights =True,
                                               verbose=1)
callbacks=[earlystopping]

In [ ]:
model_resnet.compile(loss="categorical_crossentropy",
                     optimizer=Adam(),
                     metrics=['accuracy'])

In [ ]:
history_resnet = model_resnet.fit(train_data,steps_per_epoch=len(train_data),
                    epochs=10,
                    validation_data=test_data,
                    validation_steps=len(test_data),
                    callbacks=callbacks,
                    batch_size=32)

In [ ]:
IMG_SHAPE=(224,224)
model_resnet1=tf.keras.models.Sequential([
                         hub.KerasLayer(resnet_url,
                                        trainable=True,
                                        name="feature_extractor_layer",
                                        input_shape=IMG_SHAPE+(3,)),
                         Dense(5,activation="softmax",name="output_layer")
])

In [ ]:
earlystopping=tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                               min_delta =1e-4,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights =True,
                                               verbose=1)
callbacks=[earlystopping]

In [ ]:
model_resnet1.compile(loss="categorical_crossentropy",
                     optimizer=Adam(),
                     metrics=['accuracy'])

In [ ]:
history_resnet1 = model_resnet1.fit(train_data,steps_per_epoch=len(train_data),
                    epochs=30,
                    validation_data=test_data,
                    validation_steps=len(test_data),
                    callbacks=callbacks,
                    batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
l = len(history_resnet.history["loss"])+len(history_resnet1.history["loss"])
t_acc = history_resnet.history["accuracy"]+history_resnet1.history["accuracy"]
v_acc = history_resnet.history["val_accuracy"]+history_resnet1.history["val_accuracy"]
plt.plot([x for x in range(1,l+1)],t_acc,c="red")
plt.plot([x for x in range(1,l+1)],v_acc,c="green")

In [ ]:
t_loss = history_resnet.history["loss"]+history_resnet1.history["loss"]
v_loss = history_resnet.history["val_loss"]+history_resnet1.history["val_loss"]
plt.plot([x for x in range(1,l+1)],t_loss,c="red",label="Training Loss")
plt.plot([x for x in range(1,l+1)],v_loss,c="green",label="Validation Loss")
plt.legend(loc='upper right')


In [ ]:
model_resnet.predict(test_data)

In [ ]:
predicted_classes=np.argmax(model_resnet.predict(test_data, steps=test_data.n // test_data.batch_size+1), axis=1)
predicted_classes

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predicted_classes=np.argmax(model_resnet.predict(test_data, steps=test_data.n // test_data.batch_size+1), axis=-1)
true_classes=test_data.classes
#class_labels=list(test_data.class_indices.keys())

confusionmatrix=tf.math.confusion_matrix(true_classes,predicted_classes)
plt.figure(figsize=(16,16))
sns.heatmap(confusionmatrix, cmap='Blues', annot =True ,cbar=True)

print(classification_report(true_classes,predicted_classes))